# Машинное обучение ранжированию с помощью библиотеки CatBoost

В этом примере мы:
- познакомимся с библиотекой **CatBoost**
- научимся решать задачу машинного обучения ранжирования используя алгоритмы, реализованные в **CatBoost**

Подробности про формат датасета можно найти в соседнем тюториале *explore_mslr.ipynb*, поэтому дальше предполагаем что мы с ним уже знакомы.

## Библиотека CatBoost

Полезные ссылки:
- домашняя страница: https://catboost.ai/
- официальный тюториал по машинному обучению ранжированию: https://github.com/catboost/tutorials/blob/master/ranking/ranking_tutorial.ipynb
- полный список доступных для оптимизации метрик и лоссов: https://catboost.ai/en/docs/concepts/loss-functions-ranking

## Пререквизиты

Импортируем все что нам понадобится для дальнейшей работы:

In [1]:
from timeit import default_timer as timer
import copy

import catboost
from catboost import datasets, utils

## Датасет MSLR (Microsoft Learning to Rank)

Загрузим встроенный в *catboost* сабсет датасета MSLR:

In [2]:
df_train, df_test = datasets.msrank_10k()

Посмотрим на данные:

In [3]:
df_train.head(5)

,0,1,2,3,4,5,6,7,8,9,...,128,129,130,131,132,133,134,135,136,137
0,2.0,1,3,3,0,0,3,1.0,1.0,0.000000,...,62,11089534,2,116,64034,13,3,0,0,0.0
1,2.0,1,3,0,3,0,3,1.0,0.0,1.000000,...,54,11089534,2,124,64034,1,2,0,0,0.0
2,0.0,1,3,0,2,0,3,1.0,0.0,0.666667,...,45,3,1,124,3344,14,67,0,0,0.0
3,2.0,1,3,0,3,0,3,1.0,0.0,1.000000,...,56,11089534,13,123,63933,1,3,0,0,0.0
4,1.0,1,3,0,3,0,3,1.0,0.0,1.000000,...,64,5,7,256,49697,1,13,0,0,0.0


Для удобства присвоим колонкам говорящие имена:

In [4]:
def generate_column_names(num_features):
    """Generates column names for LETOR-like datasets"""
    columns = ['label', 'qid']
    for i in range(num_features):
        column = f"feature_{i+1}"
        columns.append(column)
    return columns

# Assign column names
columns = generate_column_names(num_features=136)
df_train.columns = columns
df_test.columns = columns

Теперь наши данные выглядят понятнее:

In [5]:
df_train.head(5)

,label,qid,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_127,feature_128,feature_129,feature_130,feature_131,feature_132,feature_133,feature_134,feature_135,feature_136
0,2.0,1,3,3,0,0,3,1.0,1.0,0.000000,...,62,11089534,2,116,64034,13,3,0,0,0.0
1,2.0,1,3,0,3,0,3,1.0,0.0,1.000000,...,54,11089534,2,124,64034,1,2,0,0,0.0
2,0.0,1,3,0,2,0,3,1.0,0.0,0.666667,...,45,3,1,124,3344,14,67,0,0,0.0
3,2.0,1,3,0,3,0,3,1.0,0.0,1.000000,...,56,11089534,13,123,63933,1,3,0,0,0.0
4,1.0,1,3,0,3,0,3,1.0,0.0,1.000000,...,64,5,7,256,49697,1,13,0,0,0.0


Теперь нам надо представить датасет в формате, который можно подавать на вход катбустовой модели.  
Для этого придется разделить его на 3 части:

- **y** -- вектор таргетов
- **X** -- тензор из фичей
- **q** -- вектор из ID запросов, которые позволяют сгруппировать все документы, которые относятся к одному и тому же запросу

Важная деталь: *catboost* требует, чтобы в векторе **q** одинаковые ID запроса шли подряд (но в отличие от, например, *xgboost*, не требует их строгой сортированности). Однако в нашем случае никаких дополнительных действий не потребуется т.к. датасет уже и так отсортирован по qid.

Убедимся в этом:

In [6]:
print(df_train['qid'].is_monotonic_increasing)
print(df_test['qid'].is_monotonic_increasing)

True
True


Сконвертируем датасет в нужный формат:

In [7]:
def to_catboost_dataset(df):
    y = df['label'].to_numpy()                       # Label: [0-4]
    q = df['qid'].to_numpy()                         # Query Id
    X = df.drop(columns=['label', 'qid']).to_numpy() # 136 features
    return (X, y, q)

X_train, y_train, q_train = to_catboost_dataset(df_train)
X_test, y_test, q_test = to_catboost_dataset(df_test)

print(f"Train: X_train.shape = {X_train.shape} y_train.shape = {y_train.shape} q_train.shape = {q_train.shape}")
print(f"Test: X_test.shape = {X_test.shape} y_test.shape = {y_test.shape} q_test.shape = {q_test.shape}")

Train: X_train.shape = (10000, 136) y_train.shape = (10000,) q_train.shape = (10000,)
Test: X_test.shape = (10000, 136) y_test.shape = (10000,) q_test.shape = (10000,)


## Обучаем pointwise модель

Теперь можно приступить непосредственно к обучению модели. Мы начнем с простой _pointwise_ модели которая в качестве лосса использует обычное RMSE.

Подготовим пулы катбуста:

In [8]:
pool_train = catboost.Pool(data=X_train, label=y_train, group_id=q_train)
pool_test = catboost.Pool(data=X_test, label=y_test, group_id=q_test)

Зададим целевую метрику, которую будем оптимизировать.  
В нашем случае будем использовать NDCG@10:

In [9]:
EVAL_METRIC = 'NDCG:top=10;type=Exp'

Подготовим параметры обучения модели, в т.ч.:
- целевую метрику
- сид генератора случайных чисел
- число итераций, после которого останавливаем обучение если в течение данного числа итераций мы не наблюдаем улучшения целевой метрики на валидационном множестве

Если хотим обучаться на GPU, то еще надо добавить параметр *task_type=GPU*

In [10]:
DEFAULT_PARAMS = {
    'iterations': 1000,            # maximum possible number of trees
    'early_stopping_rounds': 100,  # stop if metric does not improve for N rounds
    'eval_metric': EVAL_METRIC,    # # metric used for early stopping
    'random_seed': 22,
    'verbose': 10
}

Мы будем обучать разные модели, использующие разные лоссы, соответствующие разным алгоритмам машинного обучения ранжированию.  
Напишем функцию, которая позволит кастомизировать модель под нужный лосс:

In [11]:
def create_model(loss_function):
    params = copy.deepcopy(DEFAULT_PARAMS)

    # Temporary directory that is used by catboost to store additional information
    catboost_info_dir = f"/tmp/catboost_info.{loss_function.lower()}"

    params.update({
        'loss_function': loss_function,
        'train_dir': str(catboost_info_dir),
    })
    return catboost.CatBoost(params)

Создадим модель:

In [12]:
model = create_model('RMSE')
print(model)

И зафитим ее на нашем обучающем множестве.

Количество деревьев будет выбрано автоматически с использованием т.н. _early stopping_ -- процесс обучения будем остановлен после того, как на валидационном множестве перестанет расти наша целевая метрика (т.е. NDCG). Обратите внимание что в данном случае целевая метрика (NDCG) отличается от лосса, который мы оптимизируем (RMSE).

Для этого передадим в функцию _fit()_ в качестве валидационного множества (параметр _eval_set_) наш тест-сет (строго говоря так делать нельзя, но мы в данном примере не стремимся к строгости, подробнее этот момент расписан в тюториале про машинное обучение ранжированию с помощью *xgboost*)

In [13]:
# Fit
start = timer()
model.fit(pool_train, eval_set=pool_test, use_best_model=True)
elapsed = timer() - start
print(f"Model fit: num_trees = {model.tree_count_} elapsed = {elapsed:.3f}")

Learning rate set to 0.073096
0:	test: 0.2493536	best: 0.2493536 (0)	total: 60.4ms	remaining: 1m
10:	test: 0.3825128	best: 0.3886914 (9)	total: 162ms	remaining: 14.6s
20:	test: 0.3867278	best: 0.3886914 (9)	total: 262ms	remaining: 12.2s
30:	test: 0.3896056	best: 0.3896056 (30)	total: 352ms	remaining: 11s
40:	test: 0.3973093	best: 0.3983815 (39)	total: 446ms	remaining: 10.4s
50:	test: 0.4030989	best: 0.4063283 (46)	total: 535ms	remaining: 9.96s
60:	test: 0.4085673	best: 0.4085673 (60)	total: 624ms	remaining: 9.61s
70:	test: 0.4053967	best: 0.4108300 (65)	total: 709ms	remaining: 9.28s
80:	test: 0.4091555	best: 0.4134532 (75)	total: 797ms	remaining: 9.04s
90:	test: 0.4050574	best: 0.4134532 (75)	total: 885ms	remaining: 8.84s
100:	test: 0.4087148	best: 0.4134532 (75)	total: 972ms	remaining: 8.65s
110:	test: 0.4114762	best: 0.4134532 (75)	total: 1.06s	remaining: 8.51s
120:	test: 0.4098458	best: 0.4140502 (112)	total: 1.15s	remaining: 8.35s
130:	test: 0.4117132	best: 0.4140502 (112)	total: 1

Видим, что модель состоит из 308 деревьев, и лучший скор NDCG@10 на тесте равен **0.419**

При желании, мы теперь можем сохранить модель в формате _cbm_:

In [14]:
model_file = "/tmp/model.cbm"

# Save model
model.save_model(model_file)

# Load model
# model = catboost.CatBoost()
# model.load_model(model_file)

Получим предикты модели на тестовом множестве:

In [15]:
y_hat_test = model.predict(pool_test)
print(f"Predicted: y_hat_test.shape = {y_hat_test.shape}")

Predicted: y_hat_test.shape = (10000,)


Теперь, имея предикты, можно посчитать метрики качества:

In [16]:
def compute_metrics(y_true, y_hat, q):
    # List of metrics to evaluate
    eval_metrics = ['NDCG:top=10;type=Exp', 'DCG:top=10;type=Exp']
    
    for eval_metric in eval_metrics:
        scores = utils.eval_metric(y_true, y_hat, eval_metric, group_id=q)
    
        # Print scores
        print(f"Evaluated: metric = {eval_metric} score = {scores[0]:.3f}")
    
# Compute metrics on test
compute_metrics(y_test, y_hat_test, q_test)

Evaluated: metric = NDCG:top=10;type=Exp score = 0.419
Evaluated: metric = DCG:top=10;type=Exp score = 8.600


Мы видим, что значение NDCG@10 на тесте совпало с тем, что вывел сам катбуст во время обучения модели!

## Обучаем LambdaMART

Теперь проделаем все то же самое, но на этот раз с использованием алгоритма *LambdaMART*:

In [17]:
# Create model
model = create_model('LambdaMart')

# Fit
start = timer()
model.fit(pool_train, eval_set=pool_test, use_best_model=True)
elapsed = timer() - start
print(f"Model fit: elapsed = {elapsed:.3f} num_trees = {model.tree_count_}")

# Predict
y_hat_test = model.predict(pool_test)
print(f"Predicted: y_hat_test.shape = {y_hat_test.shape}")

# Compute metrics on test
compute_metrics(y_test, y_hat_test, q_test)

0:	test: 0.1239106	best: 0.1239106 (0)	total: 34.2ms	remaining: 34.1s
10:	test: 0.3936757	best: 0.3936757 (10)	total: 263ms	remaining: 23.7s
20:	test: 0.4075322	best: 0.4075322 (20)	total: 487ms	remaining: 22.7s
30:	test: 0.3988492	best: 0.4105359 (21)	total: 712ms	remaining: 22.3s
40:	test: 0.4038508	best: 0.4105359 (21)	total: 932ms	remaining: 21.8s
50:	test: 0.4048661	best: 0.4105359 (21)	total: 1.15s	remaining: 21.5s
60:	test: 0.4003526	best: 0.4105359 (21)	total: 1.37s	remaining: 21.1s
70:	test: 0.4028486	best: 0.4105359 (21)	total: 1.6s	remaining: 20.9s
80:	test: 0.4030617	best: 0.4105359 (21)	total: 1.83s	remaining: 20.8s
90:	test: 0.4049737	best: 0.4105359 (21)	total: 2.05s	remaining: 20.4s
100:	test: 0.4046073	best: 0.4105359 (21)	total: 2.25s	remaining: 20s
110:	test: 0.4087473	best: 0.4105359 (21)	total: 2.46s	remaining: 19.7s
120:	test: 0.4101227	best: 0.4122323 (116)	total: 2.66s	remaining: 19.4s
130:	test: 0.4122547	best: 0.4122547 (130)	total: 2.87s	remaining: 19s
140:	t

Видно, что теперь модель обучается значительно дольше, но зато и результат получился лучше.

## Обучаем YetiRank

И, наконец, повторим все то же самое, но на этот раз с использованием алгоритма *YetiRank*:

In [18]:
# Create model
model = create_model('YetiRank')

# Fit
start = timer()
model.fit(pool_train, eval_set=pool_test, use_best_model=True)
elapsed = timer() - start
print(f"Model fit: elapsed = {elapsed:.3f} num_trees = {model.tree_count_}")

# Predict
y_hat_test = model.predict(pool_test)
print(f"Predicted: y_hat_test.shape = {y_hat_test.shape}")

# Compute metrics on test
compute_metrics(y_test, y_hat_test, q_test)

0:	test: 0.2873437	best: 0.2873437 (0)	total: 37.1ms	remaining: 37.1s
10:	test: 0.3811494	best: 0.3828752 (9)	total: 273ms	remaining: 24.5s
20:	test: 0.3886236	best: 0.3914324 (15)	total: 519ms	remaining: 24.2s
30:	test: 0.3891378	best: 0.3914324 (15)	total: 755ms	remaining: 23.6s
40:	test: 0.3946586	best: 0.3954174 (34)	total: 971ms	remaining: 22.7s
50:	test: 0.4024181	best: 0.4024181 (50)	total: 1.2s	remaining: 22.4s
60:	test: 0.4049030	best: 0.4049030 (60)	total: 1.43s	remaining: 22s
70:	test: 0.4146104	best: 0.4151574 (69)	total: 1.65s	remaining: 21.6s
80:	test: 0.4153313	best: 0.4167823 (73)	total: 1.88s	remaining: 21.3s
90:	test: 0.4221178	best: 0.4221178 (90)	total: 2.1s	remaining: 21s
100:	test: 0.4244372	best: 0.4253389 (96)	total: 2.33s	remaining: 20.7s
110:	test: 0.4244111	best: 0.4262510 (102)	total: 2.55s	remaining: 20.4s
120:	test: 0.4231627	best: 0.4273284 (117)	total: 2.77s	remaining: 20.1s
130:	test: 0.4235735	best: 0.4273284 (117)	total: 2.99s	remaining: 19.8s
140:	te

Мы получили очень впечатляющий результат!

## Результаты

Сравним результаты:

- RMSE модель выбила NDCG@10 = **0.419** (обратите внимание, насколько это лучше чем **0.34** которые выбила простая линейная регрессия из *sklearn*)
- LambdaRank выбил NDCG@10 = **0.421**
- а YetiRank выбивает уже NDCG@10 = **0.439**!

Таким образом мы наглядно видим преимущество _pairwise/listwise_-подходов над "наивным" _pointwise_-подходом.

Также, обратим внимание, что, несмотря на то, что *pairwise* подходы опять победили, абсолютные значения NDCG@10 даже у простой регрессии лучше того, что выбивала реализация LambdaRank в *хgboost* и *lightgbm*!

Видно, что "из коробки" *catboost* показывает результаты значительно лучше, чем конкуренты.  
Справедливости ради заметим, что мы в наших примерах совсем не пытались тюнить гиперпараметры -- если это проделать, то из *xgboost* и *lightgbm* можно "выжать" скоры значительно выше.  
Тем не менее, по нашему опыту, на данный момент *catboost* действительно является лучшим инструментом для решения задачи ранжирования.